In [265]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

# Prepare Data

In [266]:
# read data
spectrum_train = pd.read_excel('spectrum_train.xlsx')
spectrum_test = pd.read_excel('spectrum_valid.xlsx')
temp_train = pd.read_excel('temp_train.xlsx')
temp_test = pd.read_excel('temp_valid.xlsx')

In [267]:
# save spectrum labels for later
spectrum_labels = spectrum_test.columns

In [268]:
# scale spectrum data
scale = 10**12
spectrum_train = spectrum_train.multiply(scale)
spectrum_test = spectrum_test.multiply(scale)

In [269]:
# determine direction and set train and test data
# 1 - temp to spectrum
# 0 - spectrum to temp
direction = 0

if direction:
    X_train = temp_train
    X_test = temp_test
    y_train = spectrum_train
    y_test = spectrum_test
else:
    X_train = spectrum_train
    X_test = spectrum_test
    y_train = temp_train
    y_test = temp_test

In [270]:
# determine the supported device
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

# convert a df to tensor
def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df.values.astype(np.float32)).to(device)

In [271]:
# create input and output tensors
X = df_to_tensor(X_train)
y = df_to_tensor(y_train)

In [272]:
n_samples, input_size = X.shape
_, output_size = y.shape

# Design Model 
(input size, output size, forward pass)

In [273]:
class Net(nn.Module):
    def __init__(self, input_size, output_size):
        super(Net, self).__init__()
        self.lin1 = nn.Linear(input_size, output_size)

    def forward(self, x):
        output = self.lin1(x)
        return output

In [274]:
model = Net(input_size, output_size)

0.03583879768848419
0.10737551748752594


# Construct Loss and Optimizer

In [275]:
learning_rate = 0.01
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Training Loop

In [276]:
num_epochs = 100
for epoch in range(num_epochs):
    # forward pass and loss
    y_predicted = model(X)
    loss = criterion(y, y_predicted)

    # backward pass
    loss.backward()

    # update
    optimizer.step()

    # empty gradients
    optimizer.zero_grad()

    if (epoch+1) % 10 == 0:
        [w, b] = model.parameters()
        print(f'epoch: {epoch+1}, w = {w[0][0].item():.3f}, loss = {loss:.4f}')

epoch: 10, w = -8940214550528.000, loss = 591884225548883685576078786560.0000
epoch: 20, w = -435042847146025426637291520.000, loss = inf
epoch: 30, w = nan, loss = nan
epoch: 40, w = nan, loss = nan
epoch: 50, w = nan, loss = nan
epoch: 60, w = nan, loss = nan
epoch: 70, w = nan, loss = nan
epoch: 80, w = nan, loss = nan
epoch: 90, w = nan, loss = nan
epoch: 100, w = nan, loss = nan
